In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv('cleaneddata.csv',index_col=False)
d=pd.read_csv('smartphones.csv')
col=df.columns
col

Index(['Unnamed: 0', 'Brand', 'price', 'rating', 'bat_cap', 'bat_speed', 'ram',
       'rom', 'os_ver', 'os_type', 'inches', 'refresh_rate', 'notch', 'pixel',
       'p_cam', 's_cam', 't_cam', 'f_cam', 'proc_type', 'proc_version', 'core',
       'ghz', '5G', 'IR Blaster', 'NFC'],
      dtype='object')

In [2]:
df.drop(columns=[col[0],col[19]],inplace=True)
df.columns

Index(['Brand', 'price', 'rating', 'bat_cap', 'bat_speed', 'ram', 'rom',
       'os_ver', 'os_type', 'inches', 'refresh_rate', 'notch', 'pixel',
       'p_cam', 's_cam', 't_cam', 'f_cam', 'proc_type', 'core', 'ghz', '5G',
       'IR Blaster', 'NFC'],
      dtype='object')

In [3]:
df['p_cam'].fillna(0,inplace=True)
df['s_cam'].fillna(0,inplace=True)
df['t_cam'].fillna(0,inplace=True)
df['f_cam'].fillna(df.f_cam.mean(),inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         982 non-null    object 
 1   price         982 non-null    int64  
 2   rating        877 non-null    float64
 3   bat_cap       973 non-null    float64
 4   bat_speed     763 non-null    float64
 5   ram           982 non-null    int64  
 6   rom           982 non-null    int64  
 7   os_ver        868 non-null    float64
 8   os_type       982 non-null    object 
 9   inches        973 non-null    float64
 10  refresh_rate  610 non-null    float64
 11  notch         572 non-null    object 
 12  pixel         973 non-null    float64
 13  p_cam         982 non-null    float64
 14  s_cam         982 non-null    float64
 15  t_cam         982 non-null    float64
 16  f_cam         982 non-null    float64
 17  proc_type     978 non-null    object 
 18  core          967 non-null    

In [4]:
df.core=np.where(df['core'].str.contains('GHz'),np.NaN,df['core'])
df.core=df.core.replace(['Octa','Hexa','Quad','Dual'],[8,6,4,2])
df.core.value_counts()

8.0    884
6.0     37
4.0     33
2.0      1
Name: core, dtype: int64

In [5]:
#df=df[~df['proc_type'].str.contains('Dual')]
df=df[~df['proc_type'].isin(['Dual','Octa','Quad'])]
df.proc_type.unique()

array(['Snapdragon', 'Exynos', 'Dimensity', 'Bionic', 'Helio', 'A13', nan,
       'Unisoc', 'Tiger', 'Google', 'Apple', 'SC9863A', 'Qualcomm',
       'Spreadtrum', 'Sanpdragon', 'Fusion', 'Kirin', '(28', 'Samsung'],
      dtype=object)

In [6]:
df.proc_type=np.where(df['proc_type'].str.contains('Apple'),'Bionic',df['proc_type'])
df.proc_type=np.where(df['proc_type'].str.contains('A13'),'Bionic',df['proc_type'])
df.proc_type=np.where(df['proc_type'].str.contains('Qualcomm') | df['proc_type'].str.contains('Sanpdragon'),'Snapdragon',df['proc_type'])
df.proc_type=np.where(df['proc_type'].str.contains('SC986'),'Unisoc',df['proc_type'])
df.proc_type=np.where(df['proc_type'].str.contains('Samsung') | df['proc_type'].str.contains('28'),'Exynos',df['proc_type'])
df.proc_type.value_counts()


Snapdragon    412
Helio         201
Dimensity     177
Exynos         51
Bionic         47
Unisoc         28
Tiger          24
Google          9
Kirin           7
Spreadtrum      4
Fusion          1
Name: proc_type, dtype: int64

In [7]:

df.notch=df.notch.replace(['Dual','Small','Large'],['Punch','Notch','Notch'])
df.notch.unique()

array(['Punch', 'Water', nan, 'Notch'], dtype=object)

In [8]:
df.os_type=np.where(df['os_type'].str.contains('Memory'),np.NaN,df['os_type'])
df.os_type=np.where(df['os_type'].str.contains('Bluetooth'),np.NaN,df['os_type'])
df.os_type.unique()

array(['Android', 'iOS', 'HarmonyOS', nan, 'Pragati', 'Harmony',
       'Hongmeng', 'EMUI'], dtype=object)

In [9]:
df=df[df.price < 500000]
df

,Brand,price,rating,bat_cap,bat_speed,ram,rom,os_ver,os_type,inches,...,p_cam,s_cam,t_cam,f_cam,proc_type,core,ghz,5G,IR Blaster,NFC
0,OnePlus,54999,89.0,5000.0,100.0,12,256,13.0,Android,6.70,...,50.0,48.0,32.0,48.0,Snapdragon,8.0,3.20,1,0,1
1,OnePlus,19989,81.0,5000.0,33.0,6,128,12.0,Android,6.59,...,64.0,2.0,2.0,2.0,Snapdragon,8.0,2.20,1,0,0
2,Samsung,16499,75.0,5000.0,15.0,4,64,13.0,Android,6.60,...,50.0,2.0,2.0,2.0,Exynos,8.0,2.40,1,0,0
3,Motorola,14999,81.0,5000.0,NaN,6,128,12.0,Android,6.55,...,50.0,8.0,2.0,8.0,Snapdragon,8.0,2.20,1,0,0
4,Realme,24999,82.0,5000.0,67.0,6,128,13.0,Android,6.70,...,108.0,8.0,2.0,8.0,Dimensity,8.0,2.60,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,Vivo,119990,NaN,4800.0,66.0,12,256,12.0,Android,8.03,...,50.0,50.0,10.0,50.0,Snapdragon,8.0,3.20,1,0,1
977,Motorola,34990,83.0,5000.0,NaN,8,128,12.0,Android,6.67,...,64.0,8.0,2.0,8.0,Snapdragon,8.0,3.00,1,0,0
978,Honor,14990,75.0,5000.0,NaN,6,128,11.0,Android,6.50,...,48.0,2.0,0.0,2.0,Snapdragon,8.0,2.20,1,0,0
979,POCO,28990,85.0,5080.0,67.0,8,256,12.0,Android,6.60,...,64.0,8.0,2.0,8.0,Dimensity,8.0,2.85,1,1,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 960 entries, 0 to 980
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         960 non-null    object 
 1   price         960 non-null    int64  
 2   rating        859 non-null    float64
 3   bat_cap       951 non-null    float64
 4   bat_speed     756 non-null    float64
 5   ram           960 non-null    int64  
 6   rom           960 non-null    int64  
 7   os_ver        853 non-null    float64
 8   os_type       943 non-null    object 
 9   inches        951 non-null    float64
 10  refresh_rate  609 non-null    float64
 11  notch         572 non-null    object 
 12  pixel         951 non-null    float64
 13  p_cam         960 non-null    float64
 14  s_cam         960 non-null    float64
 15  t_cam         960 non-null    float64
 16  f_cam         960 non-null    float64
 17  proc_type     960 non-null    object 
 18  core          954 non-null    

In [11]:
SEED = 42
TARGET = 'price'
FEATURES = df.columns.drop(TARGET)

NUMERICAL = df[FEATURES].select_dtypes('number').columns
print(NUMERICAL)
print(f"Numerical features: {', '.join(NUMERICAL)}")

CATEGORICAL = pd.Index(np.setdiff1d(FEATURES, NUMERICAL))
print(CATEGORICAL)
print(f"Categorical features: {', '.join(CATEGORICAL)}")

Index(['rating', 'bat_cap', 'bat_speed', 'ram', 'rom', 'os_ver', 'inches',
       'refresh_rate', 'pixel', 'p_cam', 's_cam', 't_cam', 'f_cam', 'core',
       'ghz', '5G', 'IR Blaster', 'NFC'],
      dtype='object')
Numerical features: rating, bat_cap, bat_speed, ram, rom, os_ver, inches, refresh_rate, pixel, p_cam, s_cam, t_cam, f_cam, core, ghz, 5G, IR Blaster, NFC
Index(['Brand', 'notch', 'os_type', 'proc_type'], dtype='object')
Categorical features: Brand, notch, os_type, proc_type


In [12]:
X=df.drop(columns=TARGET)
y=df[TARGET]

In [13]:
X.isna().sum()

Brand             0
rating          101
bat_cap           9
bat_speed       204
ram               0
rom               0
os_ver          107
os_type          17
inches            9
refresh_rate    351
notch           388
pixel             9
p_cam             0
s_cam             0
t_cam             0
f_cam             0
proc_type         0
core              6
ghz              33
5G                0
IR Blaster        0
NFC               0
dtype: int64

In [14]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,VotingRegressor,ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [15]:
numerical_pipe = Pipeline([
    ('imputer',IterativeImputer()),
    ('imputer1', SimpleImputer(strategy='mean',fill_value=np.NaN)),
    ('scaler', MinMaxScaler())
])

categorical_pipe = Pipeline([

    ('imputer1', SimpleImputer(strategy='most_frequent', fill_value=np.NaN)),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False))
])

preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe, NUMERICAL),
    ('cat', categorical_pipe, CATEGORICAL)
])

pipe1 = Pipeline([
    ('preprocessors', preprocessors),
    ('model', LinearRegression())
])

pipe1.fit(X_train, y_train)
y_pred=pipe1.predict(X_test)

In [16]:
r2_score(y_test,y_pred)

-5.201298641555315e+24

In [17]:
from sklearn.ensemble import ExtraTreesRegressor
from tqdm import tqdm

In [18]:
regressor = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='friedman_mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=123, verbose=0, warm_start=False)
scores=[]
for i in tqdm(range(1000)):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe2 = Pipeline([
    ('preprocessors', preprocessors),
    ('model', regressor)
    ])
    pipe2.fit(X_train,y_train)
    y_pred=pipe2.predict(X_test)
    scores.append(r2_score(y_test,y_pred))


100%|██████████| 1000/1000 [21:19<00:00,  1.28s/it]


In [19]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
pipe2= Pipeline([
    ('preprocessors', preprocessors),
    ('model', regressor)
])
pipe2.fit(X_train,y_train)
y_pred=pipe2.predict(X_test)
r2_score(y_test,y_pred)

0.943337550777427

In [20]:
import pickle
pickle.dump(pipe2,open('ExtratreeRegressorModel.pkl','wb'))

In [ ]:
model1 = RandomForestRegressor()
model2 = LGBMRegressor()
model3 = XGBRegressor()
model4 = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='friedman_mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=42, verbose=0, warm_start=False)
model5 = AdaBoostRegressor()
model6 = GradientBoostingRegressor()
model7 = SVR()
model8 = KNeighborsRegressor()

In [ ]:
vc = VotingRegressor(estimators=[
          ('1',model1),('2',model2),('3',model3),('4',model4),('5',model5),('6',model6),('7',model7),('8',model8)])
vc = VotingRegressor(estimators=[
          ('1',model1),('4',model4),('6',model6)])

In [ ]:
X=df.drop(columns=TARGET)
y=df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
pipe1 = Pipeline([
    ('preprocessors', preprocessors),
    ('model', vc)
])

pipe1.fit(X_train, y_train)
y_pred=pipe1.predict(X_test)
r2_score(y_test,y_pred)

0.8733616548496269

In [ ]:
### Extratreeregressor works well.

In [21]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [22]:
import gradio as gr

In [ ]:
#Des="Allowed values for features:\nRating:"+'choose value any from '+str(df.rating.min())+' to '+str(df.rating.max())+'/nBattery Capacity:'+'choose value any from '+str(df.bat_cap.min())+' to '+str(df.bat_cap.max())+'\nBattery speed:'+'choose value any from  '+str(df.bat_speed.min())+' to '+str(df.bat_speed.max())"

In [23]:
inputs = [
    gr.inputs.Dropdown(list(df.Brand.unique()),label='Brand', default='Samsung'),
    gr.Markdown('Choose rating value between '+str(df.rating.min())+' and '+str(df.rating.max())),
    gr.inputs.Number(label='Rating in 0 to 100 scale', default=int(df.rating.mean())),
    gr.Markdown('Choose Battery Capacity value between '+str(df.bat_cap.min())+' and '+str(df.bat_cap.max())),
    gr.inputs.Number(label='Battery Capacity in mAH', default=int(df.bat_cap.mean())),
    gr.Markdown('Choose Battery Speed value between '+str(df.bat_speed.min())+' and '+str(df.bat_speed.max())),
    gr.inputs.Number(label='Battery charger Speed in W', default=int(df.bat_speed.mean())),
    gr.Markdown('Choose RAM value between '+str(df.ram.min())+' and '+str(df.ram.max())),
    gr.inputs.Number(label='Ram in GB', default=int(df.ram.mean())),
    gr.Markdown('Choose ROM value between '+str(df.rom.min())+' and '+str(df.rom.max())),
    gr.inputs.Number(label='Rom in GB', default=int(df.rom.mean())),
    gr.inputs.Dropdown(list(df.os_ver.unique()),label='OS Version', default='10'),
    gr.inputs.Dropdown(list(df.os_type.unique()),label='OS Type', default='Android'),
    gr.Markdown('Choose Screen Size value between '+str(df.inches.min())+' and '+str(df.inches.max())),
    gr.inputs.Number(label='Screen Size in Inches', default=df.inches.mean()),
    gr.Markdown('Choose Screen Refresh Rate value between '+str(df.refresh_rate.min())+' and '+str(df.refresh_rate.max())),
    gr.inputs.Number(label='Refresh Rate in Hz', default=int(df.refresh_rate.mean())),
    gr.inputs.Dropdown(list(df.notch.unique()),label='Notch', default='Punch'),
    gr.Markdown('Choose pixel width and height value between 250 to 3000'),
    gr.inputs.Number(label='pixel_width ', default=1080),
    gr.inputs.Number(label='pixel_height', default=1200),
    gr.Markdown('Choose primary camera value between '+str(df.p_cam.min())+' and '+str(df.p_cam.max())),
    gr.inputs.Number(label='primary_camera in MP', default=int(df.p_cam.mean())),
    gr.Markdown('Choose secondary camera value between '+str(df.s_cam.min())+' and '+str(df.s_cam.max())),
    gr.inputs.Number(label='secondary_camera in MP', default=int(df.s_cam.mean())),
    gr.Markdown('Choose teritiary camera value between '+str(df.t_cam.min())+' and '+str(df.t_cam.max())),
    gr.inputs.Number(label='Teritiary_camera in MP', default=int(df.t_cam.mean())),
    gr.Markdown('Choose front camera value between '+str(df.f_cam.min())+' and '+str(df.f_cam.max())),
    gr.inputs.Number(label='front_camera in MP', default=int(df.f_cam.mean())),
    gr.inputs.Dropdown(list(df.proc_type.unique()),label='Processor', default='Exynos'),
    gr.inputs.Dropdown(list(df.core.unique()),label='No of cores', default='8'),
    gr.Markdown('Choose clock speed in GHz value between '+str(df.ghz.min())+' and '+str(df.ghz.max())),
    gr.inputs.Number(label='Ghz', default=int(df.ghz.mean())),
    gr.inputs.Radio(label='5G',choices=['yes','no'],default='no',type='index'),
    gr.inputs.Radio(label='IR Blaster',choices=['yes','no'],default='no',type='index'),
    gr.inputs.Radio(label='NFC',choices=['yes','no'],default='no',type='index')
]

outputs = [
     gr.outputs.Label(),
     gr.outputs.Label()

]


In [24]:
def predict_price(Brand,t1,rating,t2,bat_cap,t3,bat_speed,t4,ram,t5,rom,os_ver,os_type,t6,inches,t7,refresh_rate,notch,t8,pixel_width,pixel_height,t9,p_cam,t10,s_cam,t11,t_cam,t12,f_cam,proc_type,core,t13,ghz,G5,IR_Blaster,NFC):
  pixel=int(pixel_height)*int(pixel_width)
  data=pd.DataFrame({'Brand':[Brand],'rating':[float(rating)],'bat_cap':[float(bat_cap)],'bat_speed':[float(bat_speed)],'ram':[int(ram)],'rom':[int(rom)],'os_ver':[float(os_ver)],
                     'os_type':[os_type],'inches':[float(inches)],'refresh_rate':[float(refresh_rate)],'notch':[notch],'pixel':[float(pixel)],
                     'p_cam':[float(p_cam)],'s_cam':[float(s_cam)],'t_cam':[float(t_cam)],'f_cam':[float(f_cam)],'proc_type':[proc_type],
                     'core':[float(core)],'ghz':[float(ghz)],'5G':[int(G5)],'IR Blaster':[int(IR_Blaster)],'NFC':[int(NFC)]
                   })
  X_pred=data[data.columns]
  y_pred=pipe2.predict(X_pred)[0]
  #y_pred=pipe1.predict(X_pred)[0]
  return 'Estimated price',y_pred


In [ ]:
app = gr.Interface(fn=predict_price,inputs=inputs, outputs=outputs, css="footer {visibility: hidden}", title='Mobile Price Prediction')
app.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://315439e123ab116d8a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
